# Dynamic plots (Python)

In [16]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# import dash_table_experiments as dt

import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.tools as pytools
import flask
import os

import pandas as pd
import numpy as np
from textwrap import dedent as s

In [18]:
# get data
df = pd.read_csv('shifts.csv')

In [19]:
#df.head(10)

In [20]:
import plotly.figure_factory as ff

In [21]:
countries = df.ctry.unique()
label_countries = [w.replace('_', ' ') for w in countries]
dict_countries = dict(zip(countries, label_countries))

In [22]:
df.head()

,ctry,year,segment,sex,pred_shift
0,Argentina,1950,<1950,total,8.053495
1,Argentina,1950,<1950,total,6.348885
2,Argentina,1950,<1950,total,7.212220
3,Argentina,1950,<1950,total,7.631859
4,Argentina,1950,<1950,total,7.880103


In [23]:
x1 = df.loc[(df.ctry=='Argentina') & (df.year==1950) & (df.segment=='<1950') & (df.sex=='total'), 'pred_shift'].values

In [24]:
x2 = df.loc[(df.ctry=='Argentina') & (df.year==1950) & (df.segment=='1950-1969') & (df.sex=='total'), 'pred_shift'].values
x3 = df.loc[(df.ctry=='Argentina') & (df.year==1950) & (df.segment=='1970-1989') & (df.sex=='total'), 'pred_shift'].values
x4 = df.loc[(df.ctry=='Argentina') & (df.year==1950) & (df.segment=='>=1990') & (df.sex=='total'), 'pred_shift'].values

In [25]:
hist_data = [x1,x2,x3,x4]
group_labels = ['<1950', '1950-1969', '1970-1989', '>=1990']
colors = ['#e34a33', '#2b8cbe', '#31a354', '#fdae6b']


fig = ff.create_distplot(hist_data, group_labels=group_labels,
                         curve_type='kde', show_hist=False, show_rug=False, 
                         colors=colors)

fig['layout'].update(legend=dict(orientation='h', x=0.32, y=1.1, traceorder='normal'))
fig['layout'].update(xaxis=dict(range=[np.concatenate(hist_data).min(), np.concatenate(hist_data).max()], ))
fig['layout'].update(yaxis=dict(range=[0,1]))

# Add title
fig['layout'].update(title='Expected Shift X in Year Y by Year Segment')

# Plot!
py.iplot(fig, filename='Curve and Rug')

In [68]:
# using a loop

segments = df.loc[df.ctry=='Argentina', 'segment'].unique().tolist()

In [104]:
hist_data = []

for i in segments: 
    hist_data.append(df.loc[(df.ctry=='Argentina') & (df.year==1950) &
                            (df.segment==i), 'pred_shift'].values.tolist())

In [105]:
segments

['<1950', '1950-1969', '1970-1989', '>=1990']

In [101]:
dict_colors = dict(zip(['<1950', '1950-1969', '1970-1989', '>=1990'],
                       ['#e34a33', '#2b8cbe', '#31a354', '#fdae6b']))

In [108]:
colors = []
for i in segments:
    colors.append(dict_colors[i])

In [109]:
colors

['#e34a33', '#2b8cbe', '#31a354', '#fdae6b']

In [99]:
fig = ff.create_distplot(hist_data,
        group_labels=group_labels,
        curve_type='kde',
        show_hist=False,
         show_rug=False,
         colors=colors)

In [62]:
all_options = dict()

for c in countries: 
    l = df.loc[(df.ctry==c) & (df.sex=='total'), 'year'].unique().tolist()
    s = df.loc[(df.ctry==c) & (df.sex=='total'), 'segment'].unique().tolist()
    
    td = {
        c : {'year':l, 'segment':s}
    } 
    
    all_options.update(td)

In [3]:
pred = pd.read_csv('pred.csv')

In [110]:
dt = pred.loc[pred.ctry=='Argentina']

upper_bound = go.Scatter(
    name='Upper Bound',
    x=dt['year'],
    y=dt['hi'],
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

points = go.Scatter(
    name='Observed LE',
    x=dt['year'],
    y=dt['le'],
    mode='markers', 
    marker=dict(opacity=0.5))

trace = go.Scatter(
    name='Predicted LE',
    x=dt['year'],
    y=dt['m'],
    mode='lines',
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

lower_bound = go.Scatter(
    name='Lower Bound',
    x=dt['year'],
    y=dt['lo'],
    marker=dict(color="444"),
    line=dict(width=0),
    mode='lines')

# Trace order can be important
# with continuous error bars
data = [lower_bound, trace, upper_bound, points]

layout = go.Layout(
    yaxis=dict(title='LE'),
    xaxis=dict(title='Year'),
    title='Observed vs Predicted LE',
    showlegend = False)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='le')